In [35]:
import glob
import os
import pandas as pd

In [36]:
caminho_raw = "../dados/raw/"

arquivos_bacen_resumo = "../dados/bacen/resumo_bacen/resumo_*.csv"
arquivos_bacen_dre = "../dados/bacen/dre_bacen/dre_*.csv"

lista_arquivo_resumo = glob.glob(arquivos_bacen_resumo)
lista_arquivo_dre = glob.glob(arquivos_bacen_dre)

In [37]:
lista_resumo = []

for arquivo_resumo in lista_arquivo_resumo:
    temp_df_resumo = pd.read_csv(arquivo_resumo, sep=";", encoding='utf-8', low_memory=False)
    lista_resumo.append(temp_df_resumo)

df_resumo_total = pd.concat(lista_resumo, ignore_index=True)

df_resumo_total.columns = df_resumo_total.columns.str.strip()

In [38]:
df_resumo = df_resumo_total.copy()

df_resumo['Carteira de Crédito'] = df_resumo['Carteira de Crédito'].fillna(df_resumo['Carteira de Crédito Classificada'])
df_resumo['Passivo Exigível'] = df_resumo['Passivo Exigível'].fillna(df_resumo['Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros'])

colunas_excluir = ['Carteira de Crédito Classificada','Conglomerado Prudencial.1', 'Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros', 'Títulos e Valores Mobiliários', 'Unnamed: 19', 'Unnamed: 20', 'Conglomerado Financeiro']

df_resumo = df_resumo.drop(columns=colunas_excluir)

df_resumo = df_resumo.dropna(subset='Cidade')

df_resumo.to_parquet(os.path.join(caminho_raw, "resumo.parquet"), index=False)

In [39]:
lista_dre = []

for arquivo_dre in lista_arquivo_dre:
    temp_df_dre = pd.read_csv(arquivo_dre, sep=";", encoding='utf-8', low_memory=False)
    lista_dre.append(temp_df_dre)

df_dre_total = pd.concat(lista_dre, ignore_index=True)

df_dre_total.columns = df_dre_total.columns.str.strip()

In [40]:
df_dre = df_dre_total.copy()

df_dre['Resultado Antes Impostos'] = df_dre['Resultado antes da Tributação e Participações (x) = (k) + (l) + (w)'].fillna(df_dre['Resultado antes da Tributação, Lucro e Participação (g) = (e) + (f)'])

colunas_vazias = [c for c in df_dre.columns if c.startswith('Unnamed')]

colunas_excluir = ['Conglomerado Financeiro', 'Conglomerado Prudencial.1', 'Ativo Total', 'Resultado antes da Tributação e Participações (x) = (k) + (l) + (w)', 'Resultado antes da Tributação, Lucro e Participação (g) = (e) + (f)']

colunas_exclusão = colunas_vazias + colunas_excluir

df_dre = df_dre.dropna(subset='Cidade')

df_dre = df_dre.drop(columns=colunas_exclusão)

df_dre.to_parquet(os.path.join(caminho_raw, "dre.parquet"), index=False)